In [1]:
%env WORKDIR=/tmp/vault
%env VAULT_K8S_NAMESPACE=vault
%env VAULT_HELM_RELEASE_NAME=vault
%env VAULT_SERVICE_NAME=vault-internal 
%env K8S_CLUSTER_NAME=cluster.local 

env: WORKDIR=/tmp/vault
env: VAULT_K8S_NAMESPACE=vault
env: VAULT_HELM_RELEASE_NAME=vault
env: VAULT_SERVICE_NAME=vault-internal
env: K8S_CLUSTER_NAME=cluster.local


In [2]:
%%bash
cat > ${WORKDIR}/overrides_csi.yaml <<EOF
global:
   enabled: true
   tlsDisable: false # Disabling TLS to avoid issues when connecting to Vault via port forwarding

csi:
  enabled: true
  image:
    repository: hashicorp/vault-csi-provider


injector:
   enabled: false
   #repository: registry.connect.redhat.com/hashicorp/vault-k8s
   # Testing on Mac M1 so moving to arm image
   image:
      repository: docker.io/hashicorp/vault-k8s
      #repository: registry.connect.redhat.com/hashicorp/vault-k8s
      # Testing on Mac M1 so moving to arm image
   agentImage:
      repository: docker.io/hashicorp/vault
      #repository: registry.connect.redhat.com/hashicorp/vault
      # Testing on Mac M1 so moving to arm image

# Supported log levels include: trace, debug, info, warn, error
logLevel: "trace" # Set to trace for initial troubleshooting, info for normal operation

server:
# config.yaml
   image:
      repository: docker.io/hashicorp/vault-enterprise
      tag: 1.13.1-ent

   enterpriseLicense:
      secretName: vault-ent-license
   extraEnvironmentVars:
      VAULT_CACERT: /vault/userconfig/vault-ha-tls/vault.ca
      VAULT_TLSCERT: /vault/userconfig/vault-ha-tls/vault.crt
      VAULT_TLSKEY: /vault/userconfig/vault-ha-tls/vault.key
   volumes:
      - name: userconfig-vault-ha-tls
        secret:
         defaultMode: 420
         secretName: vault-ha-tls
   volumeMounts:
      - mountPath: /vault/userconfig/vault-ha-tls
        name: userconfig-vault-ha-tls
        readOnly: true
   standalone:
      enabled: false
   affinity: ""
   ha:
      enabled: true
      replicas: 3
      raft:
         enabled: true
         setNodeId: true
         config: |
            ui = true
            listener "tcp" {
               tls_disable = 0 # Disabling TLS to avoid issues when connecting to Vault via port forwarding
               address = "[::]:8200"
               cluster_address = "[::]:8201"
               tls_cert_file = "/vault/userconfig/vault-ha-tls/vault.crt"
               tls_key_file  = "/vault/userconfig/vault-ha-tls/vault.key"
               tls_client_ca_file = "/vault/userconfig/vault-ha-tls/vault.ca"
               api_address = "https://vault-active.vault.svc.cluster.local:8200"
            }
            storage "raft" {
               path = "/vault/data"
            
               retry_join {
                  auto_join             = "provider=k8s namespace=vault label_selector=\"component=server,app.kubernetes.io/name=vault\""
                  auto_join_scheme      = "https"
                  leader_ca_cert_file   = "/vault/userconfig/vault-ha-tls/vault.ca"
                  leader_tls_servername = "vaultpr-0.vault-internal" #Tiene que matchear una SAN del certificado
               }
            
            }
            disable_mlock = true
            service_registration "kubernetes" {}
EOF


In [3]:
%%bash

helm upgrade -n $VAULT_K8S_NAMESPACE $VAULT_HELM_RELEASE_NAME hashicorp/vault -f ${WORKDIR}/overrides_csi.yaml 

Release "vault" has been upgraded. Happy Helming!
NAME: vault
LAST DEPLOYED: Thu Apr 27 11:31:36 2023
NAMESPACE: vault
STATUS: deployed
REVISION: 3
NOTES:
Thank you for installing HashiCorp Vault!

Now that you have deployed Vault, you should look over the docs on using
Vault with Kubernetes available here:

https://www.vaultproject.io/docs/


Your release is named vault. To learn more about the release, try:

  $ helm status vault
  $ helm get manifest vault


In [4]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-0 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Key                     Value
---                     -----
Seal Type               shamir
Initialized             true
Sealed                  false
Total Shares            1
Threshold               1
Version                 1.13.1+ent
Build Date              2023-03-23T20:09:57Z
Storage Type            raft
Cluster Name            vault-cluster-b636cc4a
Cluster ID              c1a91f42-31d2-ba5b-e4e2-2e91c445a8ed
HA Enabled              true
HA Cluster              https://vault-0.vault-internal:8201
HA Mode                 active
Active Since            2023-04-27T09:05:07.219118672Z
Raft Committed Index    790
Raft Applied Index      790
Last WAL                56


In [5]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-1 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Key                                    Value
---                                    -----
Seal Type                              shamir
Initialized                            true
Sealed                                 false
Total Shares                           1
Threshold                              1
Version                                1.13.1+ent
Build Date                             2023-03-23T20:09:57Z
Storage Type                           raft
Cluster Name                           vault-cluster-b636cc4a
Cluster ID                             c1a91f42-31d2-ba5b-e4e2-2e91c445a8ed
HA Enabled                             true
HA Cluster                             https://vault-0.vault-internal:8201
HA Mode                                standby
Active Node Address                    https://10.244.0.8:8200
Performance Standby Node               true
Performance Standby Last Remote WAL    56
Raft Committed Index                   790
Raft Applied Index                     790


In [6]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-2 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Key                                    Value
---                                    -----
Seal Type                              shamir
Initialized                            true
Sealed                                 false
Total Shares                           1
Threshold                              1
Version                                1.13.1+ent
Build Date                             2023-03-23T20:09:57Z
Storage Type                           raft
Cluster Name                           vault-cluster-b636cc4a
Cluster ID                             c1a91f42-31d2-ba5b-e4e2-2e91c445a8ed
HA Enabled                             true
HA Cluster                             https://vault-0.vault-internal:8201
HA Mode                                standby
Active Node Address                    https://10.244.0.8:8200
Performance Standby Node               true
Performance Standby Last Remote WAL    56
Raft Committed Index                   792
Raft Applied Index                     792


In [7]:
%%bash
cat ${WORKDIR}/cluster-keys.json | jq -r ".root_token"

hvs.22iw9eugNC3Kcdsv8Y9fHDHG


In [8]:
%env VAULT_ADDR=https://127.0.0.1:8200
%env VAULT_TOKEN=hvs.22iw9eugNC3Kcdsv8Y9fHDHG
%env VAULT_CACERT=/tmp/vault/vault.ca

env: VAULT_ADDR=https://127.0.0.1:8200
env: VAULT_TOKEN=hvs.22iw9eugNC3Kcdsv8Y9fHDHG
env: VAULT_CACERT=/tmp/vault/vault.ca


In [9]:
%%bash
## Create K8S Auth method with mock data
vault policy write internal-app - <<EOF
path "secret/data/*" {
  capabilities = ["read"]
}
EOF



Success! Uploaded policy: internal-app


In [10]:
%%bash
vault secrets enable -path=secret kv-v2

Success! Enabled the kv-v2 secrets engine at: secret/


In [11]:
%%bash
vault kv put secret/db-pass password="db-secret-password"
vault kv put secret/db-pass2 user="test" password="db-secret-password2"

=== Secret Path ===
secret/data/db-pass

======= Metadata =======
Key                Value
---                -----
created_time       2023-04-27T09:32:43.862826508Z
custom_metadata    <nil>
deletion_time      n/a
destroyed          false
version            1
==== Secret Path ====
secret/data/db-pass2

======= Metadata =======
Key                Value
---                -----
created_time       2023-04-27T09:32:43.950936508Z
custom_metadata    <nil>
deletion_time      n/a
destroyed          false
version            1


In [22]:
%%bash
vault write auth/kubernetes/role/database \
    bound_service_account_names=webapp-sa \
    bound_service_account_namespaces=default \
    policies=internal-app \
    ttl=20m


Success! Data written to: auth/kubernetes/role/database


In [12]:
%%bash
helm repo add secrets-store-csi-driver https://kubernetes-sigs.github.io/secrets-store-csi-driver/charts
helm install csi secrets-store-csi-driver/secrets-store-csi-driver \
    --set syncSecret.enabled=true


"secrets-store-csi-driver" already exists with the same configuration, skipping
NAME: csi
LAST DEPLOYED: Thu Apr 27 11:33:27 2023
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
The Secrets Store CSI Driver is getting deployed to your cluster.

To verify that Secrets Store CSI Driver has started, run:

  kubectl --namespace=default get pods -l "app=secrets-store-csi-driver"

Now you can follow these steps https://secrets-store-csi-driver.sigs.k8s.io/getting-started/usage.html
to create a SecretProviderClass resource, and a deployment using the SecretProviderClass.


In [17]:
%%bash
kubectl get pods

NAME                                 READY   STATUS    RESTARTS   AGE
csi-secrets-store-csi-driver-7njnt   3/3     Running   0          41s


In [48]:
%%bash
cat > spc-vault-database.yaml <<EOF
apiVersion: secrets-store.csi.x-k8s.io/v1
kind: SecretProviderClass
metadata:
  name: vault-database
spec:
  # https://developer.hashicorp.com/vault/docs/platform/k8s/csi/configurations 
  provider: vault
  parameters:
    vaultAddress: "https://vault-internal.vault.svc.cluster.local:8200"
    vaultSkipTLSVerify: "true"
    roleName: "database"
    objects: |
      - objectName: "db-password"
        secretPath: "secret/data/db-pass"
        secretKey: "password"
EOF
kubectl apply --filename spc-vault-database.yaml

secretproviderclass.secrets-store.csi.x-k8s.io/vault-database configured


In [19]:
%%bash
kubectl describe SecretProviderClass vault-database

Name:         vault-database
Namespace:    default
Labels:       <none>
Annotations:  <none>
API Version:  secrets-store.csi.x-k8s.io/v1
Kind:         SecretProviderClass
Metadata:
  Creation Timestamp:  2023-04-27T09:40:04Z
  Generation:          1
  Managed Fields:
    API Version:  secrets-store.csi.x-k8s.io/v1
    Fields Type:  FieldsV1
    fieldsV1:
      f:metadata:
        f:annotations:
          .:
          f:kubectl.kubernetes.io/last-applied-configuration:
      f:spec:
        .:
        f:parameters:
          .:
          f:objects:
          f:roleName:
          f:vaultAddress:
          f:vaultSkipTLSVerify:
        f:provider:
    Manager:         kubectl-client-side-apply
    Operation:       Update
    Time:            2023-04-27T09:40:04Z
  Resource Version:  4108
  UID:               17289105-8ead-4783-86e0-f619661e0dd1
Spec:
  Parameters:
    Objects:  - objectName: "db-password"
  secretPath: "secret/data/*"

    Role Name:              database
    Vault Addre

In [20]:
%%bash
kubectl create serviceaccount webapp-sa

serviceaccount/webapp-sa created


In [27]:
%%bash
cat > webapp-pod.yaml <<EOF
kind: Pod
apiVersion: v1
metadata:
  name: webapp
spec:
  serviceAccountName: webapp-sa
  containers:
  - image: jweissig/app:0.0.1
    name: webapp
    volumeMounts:
    - name: secrets-store-inline
      mountPath: "/mnt/secrets-store"
      readOnly: true
  volumes:
    - name: secrets-store-inline
      csi:
        driver: secrets-store.csi.k8s.io
        readOnly: true
        volumeAttributes:
          secretProviderClass: "vault-database"
EOF
kubectl apply --filename webapp-pod.yaml

pod/webapp configured


In [52]:
%%bash
kubectl exec webapp -- cat /mnt/secrets-store/db-password

db-secret-password

## Sync to Secret

In [53]:
%%bash
cat > spc-vault-database.yaml <<EOF
apiVersion: secrets-store.csi.x-k8s.io/v1
kind: SecretProviderClass
metadata:
  name: vault-database
spec:
  provider: vault
  secretObjects:
  - data:
    - key: password
      objectName: db-password
    secretName: dbpass
    type: Opaque
  parameters:
    vaultAddress: "https://vault-internal.vault.svc.cluster.local:8200"
    vaultSkipTLSVerify: "true"
    roleName: "database"
    objects: |
      - objectName: "db-password"
        secretPath: "secret/data/db-pass"
        secretKey: "password"
EOF
kubectl apply --filename spc-vault-database.yaml

secretproviderclass.secrets-store.csi.x-k8s.io/vault-database configured


In [54]:
%%bash
cat > webapp-pod.yaml <<EOF
kind: Pod
apiVersion: v1
metadata:
  name: webapp
spec:
  serviceAccountName: webapp-sa
  containers:
  - image: jweissig/app:0.0.1
    name: webapp
    env:
    - name: DB_PASSWORD
      valueFrom:
        secretKeyRef:
          name: dbpass
          key: password
    volumeMounts:
    - name: secrets-store-inline
      mountPath: "/mnt/secrets-store"
      readOnly: true
  volumes:
    - name: secrets-store-inline
      csi:
        driver: secrets-store.csi.k8s.io
        readOnly: true
        volumeAttributes:
          secretProviderClass: "vault-database"
EOF
## Delete deployment and redeploy
kubectl delete pod webapp && kubectl apply --filename webapp-pod.yaml

pod "webapp" deleted
pod/webapp created


In [55]:
%%bash
#Verify secrets is created
kubectl get secret dbpass

NAME     TYPE     DATA   AGE
dbpass   Opaque   1      18s


In [56]:
%%bash
# Check secret as environmental variable
kubectl exec webapp -- env | grep DB_PASSWORD

DB_PASSWORD=db-secret-password


In [57]:
%%bash
kubectl delete pod webapp
helm uninstall vault -n vault

pod "webapp" deleted
release "vault" uninstalled
